In [1]:
import anndata as ad
import scanpy as sc
import pandas as pd
import scipy as sp
import numpy as np
import pickle as pkl
from sklearn.cluster import KMeans
import igraph as ig
import leidenalg as la
import utils

In [2]:
adata = sc.read('./../data/MrVIoutputs/bacdrop.h5ad')


In [3]:
ranked_genes = utils.mrvi_identify_cell_states(adata, 'X_mrvi_z', neighbor_method='knn')

/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:394: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'names'] = self.var_names[global_indices]
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:396: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  self.stats[group_name, 'scores'] = scores[global_indices]
/home/dennis/miniconda3/envs/compbioGPU/lib/python3.9/site-packages/scanpy/tools/_rank_genes_groups.py:

In [4]:
# sc.pl.umap(adata, use_raw=True)
# sc.pl.embedding(adata, basis='X_mrvi_z')

In [5]:
df_og = ranked_genes
df = df_og.copy()

In [6]:
# sc.pl.umap(adata, use_raw=False)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,cds-WP_002919766.1,cds-WP_004174069.1-2,cds-WP_012068464.1,cds-WP_002920554.1,cds-WP_004174069.1,cds-WP_002898162.1,cds-WP_002919796.1,cds-WP_032430362.1,cds-WP_171290763.1,cds-WP_013815099.1-30,...,cds-WP_004145074.1,cds-WP_012068464.1,cds-WP_004174069.1-2,cds-WP_013815099.1-29,cds-WP_004174069.1-2,cds-WP_013815099.1-79,cds-WP_000242758.1,cds-WP_002919219.1,cds-WP_002919773.1,cds-WP_002910077.1
1,cds-WP_032424651.1,cds-WP_004174069.1,cds-WP_002888735.1,cds-WP_004150943.1,cds-WP_004174069.1-2,cds-WP_002920103.1,cds-WP_012068464.1,cds-WP_050484295.1,cds-WP_032429520.1,cds-WP_013815099.1-48,...,cds-WP_001181005.1,cds-WP_002898408.1,cds-WP_032429669.1,cds-WP_013815099.1-78,cds-WP_002884743.1,cds-WP_013815099.1-35,cds-WP_002889299.1,cds-WP_032430210.1,cds-WP_002887843.1,cds-WP_021440992.1
2,cds-WP_020802781.1,cds-WP_004151070.1,cds-WP_004144748.1,cds-WP_002882755.1,cds-WP_002888735.1,cds-WP_032424651.1,cds-WP_000617546.1,cds-WP_002913016.1,cds-WP_179139609.1,cds-WP_013815099.1-49,...,cds-WP_004151523.1,cds-WP_002915213.1,cds-WP_012068464.1,cds-WP_013815099.1-71,cds-WP_002895166.1,cds-WP_013815099.1-63,cds-WP_050484291.1,cds-WP_002898303.1,cds-WP_002884725.1,cds-WP_002913950.1


In [7]:
# For each column 
for c in df.columns:
    # For the entries in each column
    duplicate = None
    for i in range(len(df[c])):
        temp = df[c][i].split('-')
        
        if len(temp) > 2:
            # replace with empty string
            if duplicate and (duplicate == temp[-1]):
                df[c][i] = ''
            
            if not duplicate:
                duplicate = temp[-1]
            

In [8]:
# # df.head(20)
# # t = None
# # t = '2'
# # if 5==5 and t=='2':
#     # print("hi")

# # df['4'][1].split('-')[-1]
# # df.head(30)
# d = {}
# d['c'] = []
# d['c'].append('vubi')
# d['c'].append('vubibunin')
# len(d['c'])

In [9]:
# TODO: write only the top 30, manually create the 30-entry columns from the main df by only using the non-empty string entries!
# df.to_csv("./../out/test.csv")
# sc.pl.embedding(adata, basis='X_mrvi_u')
# d = {'col1': [1, 2], 'col2': [3, 4]}
# df = pd.DataFrame(data=d)
d = {}
for c in df.columns:
    d[c] = []
    max_itr = len(df[c])
    itr = 0
    while (max_itr > itr) and (len(d[c]) < 30):
        if df[c][itr] != '':
            d[c].append(df[c][itr])
        
        itr += 1

In [10]:
# (df['0'][0] != '')
# for key in d:
#     print(len(d[key]))
df_top = pd.DataFrame(data=d)
# df_top

In [11]:
df_test = utils.filter_duplicates(df_og, 30)

In [12]:
for c in df_test.columns:
    for i in range(len(df_test[c])):
        if df_top[c][i] != df_test[c][i]:
            raise ValueError("Function return differs from test case")

In [13]:
# df_test = pd.read_csv("./../out/test.csv")

In [14]:
# for i in range(len(df_test.columns)):
#     print(len(np.unique(df_test['0'].to_numpy())))

"Multiple values can be given to color. In the following example we will plot 6 genes: ‘CD79A’, ‘MS4A1’, ‘IGJ’, CD3D’, ‘FCER1A’, and ‘FCGR3A’ to get an idea on where those marker genes are being expressed."

In [15]:
# TODO: figure out the markers for bacdrop and plot adding them as the colour label to identify where they're being expressed 
# e.g. sc.pl.umap(pbmc, color=['CD79A', 'MS4A1', 'IGJ', 'CD3D', 'FCER1A', 'FCGR3A', 'n_counts', 'bulk_labels'])